In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
from project.models import Project
from public_data.models import Ocsge2015, CouvertureSol, UsageSol

In [3]:
raw_data = json.loads(Project.objects.get(pk=4).couverture_usage)
# raw_data

In [4]:
millesimes = raw_data.keys()
for couv in CouvertureSol.iter_total_surface(raw_data):
    print(couv.code, couv.label, end=" ")
    for m in millesimes:
        print(couv.total_surface[m], end=" ")
    print()

1 Sans végétation 25.2640886853288 26.222390498532977 
1.1 Surfaces anthropisées 22.600233146933196 23.429295638135642 
1.1.1 Zones imperméables 20.702556697746438 21.42303811952844 
1.1.1.1 Zones bâties 12.348765754474607 12.929375403898439 
1.1.1.2 Zones non bâties (Routes; places; parking…) 8.353790943271829 8.49366271563 
1.1.2 Zones perméables 1.8976764491867564 2.0062575186072036 
1.1.2.1 Zones à matériaux minéraux 1.8770371134357422 1.994255737751534 
1.1.2.2 Zones à autres matériaux composites 0.020639335751014093 0.01200178085566955 
1.2 Surfaces naturelles 2.6638555383956035 2.7930948603973347 
1.2.1 Sols nus (Sable; pierres meubles; rochers saillants; …) 0.6679596499533035 0.6705329264952924 
1.2.2 Surfaces d'eau (Eau continentale et maritime) 1.9958958884423 2.1225619339020425 
1.2.3 Névés et glaciers 0 0 
2 Avec végétation 189.3313633633219 188.59905472692225 
2.1 Végétation ligneuse 157.74658227012216 157.56907786509223 
2.1.1 Formations arborées 155.0770437963863 154.921